In [1]:
from requests_html import HTMLSession
import scrapy
from scrapy.crawler import CrawlerProcess
# import pandas as pd
# import sqlite3
# from trafilatura import extract , fetch_url
from pprint import pprint as print

In [2]:
def data_next_page_id(base_size=60, maximum_size=110):
    count = 1
    session = HTMLSession()
    next_page_ids = [0]
    while count != 0:
        new_next_page_ids = []
        for _ in next_page_ids:
            url = f'https://api.divar.ir/v8/map-discovery/bbox/posts?lon1=50.780457955008785&lat1=35.43862123757947&lon2=51.702507060611595&lat2=35.94376848564657&nextPageId={_}&filters=building-age=0-10&filters=category=apartment-sell&filters=size={base_size}-{maximum_size}'
            response = session.get(url)
            new_next_page_ids.append(response.json().get("next_page_id"))
        next_page_ids.extend(new_next_page_ids)  # Use extend() instead of append()
        count -= 1
    return list(set(next_page_ids))

In [3]:
apartment = data_next_page_id(60,110)
def tokens():
    urls=[]
    tokens=[]
    for _ in apartment:
        urls.append(f'https://api.divar.ir/v8/map-discovery/bbox/posts?lon1=50.780457955008785&lat1=35.43862123757947&lon2=51.702507060611595&lat2=35.94376848564657&nextPageId={_}&filters=building-age=0-10&filters=category=apartment-sell&filters=size=60-110')
    for url in urls:
        session = HTMLSession()
        response=session.get(url)
        datas=response.json()["posts"]
        for _ in datas:
            tokens.append(_.get("token"))
    return tokens
# tokens()

In [5]:
class Divar(scrapy.Spider):
    name="divar"
    start_urls=[f"https://divar.ir/v/{_}" for _ in tokens()]

    def parse(self, response):
        location= str(response.xpath('//*[@id="app"]/div[2]/div/div[1]/div[1]/div[1]/div[2]/div[2]/text()').extract()).split("در")[1]
        price_per_meter=str(response.xpath("//*[@id='app']/div[2]/div/div[1]/div[1]/div[4]/div[3]/div[2]/text()").extract())
        print(5*"***")
        print(price_per_meter)

if __name__=="__main__":
    p=CrawlerProcess()
    p.crawl(Divar)
    p.start()

2023-07-21 00:33:42 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
2023-07-21 00:33:42 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.2, Platform Linux-5.15.0-76-generic-x86_64-with-glibc2.35
2023-07-21 00:33:42 [scrapy.crawler] INFO: Overridden settings:
{}
2023-07-21 00:33:42 [py.warnings] WARNING: /home/dn0153/Documents/divar_scraper/.venv/lib/python3.10/site-packages/scrapy/utils/request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See 